In [8]:
# W205 Project 2
# Author: Kevin Xuan

In [ ]:
'''


'''

In [2]:
# read data
data = spark \
  .read \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka:29092") \
  .option("subscribe","ad_data") \
  .option("startingOffsets", "earliest") \
  .option("endingOffsets", "latest") \
  .load() 

In [5]:
# Cache this to Remove Warning messages which will appear later
data.cache()

DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]

In [9]:
data.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [10]:
data.show(2)

+----+--------------------+-------+---------+------+--------------------+-------------+
| key|               value|  topic|partition|offset|           timestamp|timestampType|
+----+--------------------+-------+---------+------+--------------------+-------------+
|null|[7B 22 6B 65 65 6...|ad_data|        0|     0|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|ad_data|        0|     1|1969-12-31 23:59:...|            0|
+----+--------------------+-------+---------+------+--------------------+-------------+
only showing top 2 rows



In [3]:
ad_data = data.select(data.value.cast('string'))

In [14]:
ad_data.show()

+--------------------+
|               value|
+--------------------+
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
+--------------------+
only showing top 20 rows



In [4]:
import json
ad = ad_data.rdd.map(lambda x: json.loads(x.value)).toDF()


/spark-2.2.0-bin-hadoop2.6/python/pyspark/sql/session.py:351: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


In [15]:
# save table to HDFS
# ad.write.parquet("/tmp/exam_data")

In [16]:
ad =None

In [17]:
# read file from HDFS
ad = spark.read.parquet("/tmp/exam_data")

In [18]:
ad.show(5)

+--------------------+-------------+--------------------+------------------+--------------------+------------------+------------+--------------------+--------------------+--------------------+
|        base_exam_id|certification|           exam_name|   keen_created_at|             keen_id|    keen_timestamp|max_attempts|           sequences|          started_at|        user_exam_id|
+--------------------+-------------+--------------------+------------------+--------------------+------------------+------------+--------------------+--------------------+--------------------+
|37f0a30a-7464-11e...|        false|Normal Forms and ...| 1516717442.735266|5a6745820eb8ab000...| 1516717442.735266|         1.0|Map(questions -> ...|2018-01-23T14:23:...|6d4089e4-bde5-4a2...|
|37f0a30a-7464-11e...|        false|Normal Forms and ...| 1516717377.639827|5a674541ab6b0a000...| 1516717377.639827|         1.0|Map(questions -> ...|2018-01-23T14:21:...|2fec1534-b41f-441...|
|4beeac16-bb83-4d5...|        false

In [5]:
ad.registerTempTable('exam_data')

In [27]:
spark.sql("select count(user_exam_id) from exam_data").show()

+-------------------+
|count(user_exam_id)|
+-------------------+
|               3280|
+-------------------+



In [25]:
# Question 1
spark.sql("select count(base_exam_id) from exam_data").show()

+-------------------+
|count(base_exam_id)|
+-------------------+
|               3280|
+-------------------+



In [ ]:
# 1. How many assessments are in the dataset?
# Answer: There are 3,280 assessments in the dataset

In [ ]:
# 2. What's the name of your Kafka topic? How did you come up with that name?
# A: The name of my Kafka topic is ad_data. I come up with that name because the
#    dataset we are given is around advertisements (ads), and our objective is to
#    tracker user activity with these advertisement data. 

In [29]:
# Question 3
spark.sql("select count(exam_name) from exam_data where exam_name = 'Learning Git'").show()

+----------------+
|count(exam_name)|
+----------------+
|             394|
+----------------+



In [ ]:
# 3. How many people took Learning Git?
# Answer: 394 people took Learning Git.

In [33]:
# Question 4
# Most Common Course
spark.sql(\
    "select exam_name, count(exam_name)\
    from exam_data\
    group by exam_name\
    order by count(exam_name)\
    DESC limit 1").show()

+------------+----------------+
|   exam_name|count(exam_name)|
+------------+----------------+
|Learning Git|             394|
+------------+----------------+



In [40]:
# least common course taken
spark.sql(\
    "select exam_name, count(exam_name)\
    from exam_data\
    group by exam_name\
    order by count(exam_name)\
    limit 1").show()

+--------------------+----------------+
|           exam_name|count(exam_name)|
+--------------------+----------------+
|Learning to Visua...|               1|
+--------------------+----------------+



In [ ]:
spark.sql(\
    "select exam_name\
    from exam_data\
    group by exam_name\
    order by count(exam_name)\
    limit 1").show()

In [ ]:
# 4. What is the least common course taken? And the most common?
# Answer: The least common course taken is "Learning to Visualization. 
#         The most common course taken is "Learning Git"

In [ ]:
# 5. How many courses are there in the dataset?

In [6]:
spark.sql(\
    "select count(distinct exam_name) from exam_data").show()

+-------------------------+
|count(DISTINCT exam_name)|
+-------------------------+
|                      103|
+-------------------------+



In [9]:
# 6. What's the earliest and latest date for an exam?
spark.sql(\
    "select min(started_at) as earliest_date, max(started_at) as latest_date from exam_data").show()

+--------------------+--------------------+
|       earliest_date|         latest_date|
+--------------------+--------------------+
|2017-11-21T00:42:...|2018-01-28T19:17:...|
+--------------------+--------------------+



In [ ]:
# The earliest exam is on Novemember 21st in 2017, and the latest exam is January 28th on 2018.

In [11]:
spark.sql(\
    "select extract(hour from started_at) from exam_data").show()


ParseException: "\nmismatched input 'from' expecting {')', ','}(line 1, pos 20)\n\n== SQL ==\nselect extract(hour from started_at) from ad_data\n--------------------^^^\n"